# AgentCore Memory Demo

This notebook demonstrates Amazon Bedrock AgentCore Memory with Strands agents:
- Short-term memory for immediate context
- Long-term memory with semantic, preference, and summary strategies
- Cross-session memory persistence

In [ ]:
# Install dependencies
!pip install bedrock-agentcore strands-agents -q

In [1]:
from datetime import datetime
from strands import Agent
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.integrations.strands.config import AgentCoreMemoryConfig
from bedrock_agentcore.memory.integrations.strands.session_manager import AgentCoreMemorySessionManager

## Part 1: Short-Term Memory

Basic memory for immediate conversational context

In [2]:
# Create memory client
client = MemoryClient(region_name="us-east-1")

# Create basic memory
basic_memory = client.create_memory(
    name="ShortTermDemo",
    description="Basic memory for short-term context"
)

print(f"Memory created: {basic_memory['id']}")

Memory created: ShortTermDemo-Sc32klB1dr


In [5]:
# Configure memory
MEM_ID = basic_memory['id']
ACTOR_ID = f"user_{datetime.now().strftime('%Y%m%d%H%M%S')}"
SESSION_ID = f"session_{datetime.now().strftime('%Y%m%d%H%M%S')}"

config = AgentCoreMemoryConfig(
    memory_id=MEM_ID,
    session_id=SESSION_ID,
    actor_id=ACTOR_ID
)

# Create session manager
session_manager = AgentCoreMemorySessionManager(
    agentcore_memory_config=config,
    region_name="us-east-1"
)

print(f"Session configured: {SESSION_ID}")

Session configured: session_20260204001742


In [6]:
# Create agent with memory
agent = Agent(
    system_prompt="You are a helpful assistant. Use what you know about the user to provide personalized responses.",
    session_manager=session_manager
)

print("Agent created with short-term memory")

Agent created with short-term memory


### Test Short-Term Memory

In [7]:
# Turn 1: Share information
response1 = agent("My name is Alice and I work as a software engineer")
print("Turn 1:")
print(response1)

Hello Alice! It's nice to meet you. Software engineering is such a dynamic and impactful field - there's always something new to learn and build. What kind of software development do you focus on? Are you working on web applications, mobile apps, systems programming, or something else entirely?Turn 1:
Hello Alice! It's nice to meet you. Software engineering is such a dynamic and impactful field - there's always something new to learn and build. What kind of software development do you focus on? Are you working on web applications, mobile apps, systems programming, or something else entirely?



In [8]:
# Turn 2: Agent should remember
response2 = agent("What's my name and what do I do?")
print("\nTurn 2:")
print(response2)

Your name is Alice and you work as a software engineer.
Turn 2:
Your name is Alice and you work as a software engineer.



In [9]:
# Turn 3: Build on context
response3 = agent("I'm learning Python. Can you suggest resources?")
print("\nTurn 3:")
print(response3)

That's great that you're learning Python, Alice! Given your background as a software engineer, you'll probably pick it up quickly since you already understand programming concepts. Here are some resources I'd recommend:

**For structured learning:**
- **Automate the Boring Stuff with Python** - Practical and project-focused
- **Python Crash Course** - Good balance of fundamentals and projects
- **Real Python** website - Excellent tutorials for all levels

**Practice platforms:**
- **LeetCode** (Python track) - Great for algorithm practice
- **HackerRank** - Good for both basics and advanced challenges
- **Codewars** - Fun bite-sized problems

**Documentation & Reference:**
- Official Python documentation - Always reliable
- **Python.org's tutorial** - Comprehensive and free

**Project ideas to apply your learning:**
- Web scraping tools
- Data analysis scripts
- Simple web APIs with Flask/FastAPI
- Automation scripts for daily tasks

Since you're already a software engineer, you might 

## Part 2: Long-Term Memory with All Strategies

Comprehensive memory with semantic, preference, and summary strategies

In [ ]:
# Create comprehensive memory
comprehensive_memory = client.create_memory_and_wait(
    name="ComprehensiveDemo",
    description="Full-featured memory with all strategies",
    strategies=[
        {
            "summaryMemoryStrategy": {
                "name": "SessionSummarizer",
                "namespaces": ["/summaries/{actorId}/{sessionId}/"]
            }
        },
        {
            "userPreferenceMemoryStrategy": {
                "name": "PreferenceLearner",
                "namespaces": ["/preferences/{actorId}/"]
            }
        },
        {
            "semanticMemoryStrategy": {
                "name": "FactExtractor",
                "namespaces": ["/facts/{actorId}/"]
            }
        }
    ]
)

print(f"Comprehensive memory created: {comprehensive_memory['id']}")

In [ ]:
# Configure long-term memory
LT_MEM_ID = comprehensive_memory['id']
LT_ACTOR_ID = f"user_{datetime.now().strftime('%Y%m%d%H%M%S')}"
LT_SESSION_ID = f"session_{datetime.now().strftime('%Y%m%d%H%M%S')}"

lt_config = AgentCoreMemoryConfig(
    memory_id=LT_MEM_ID,
    session_id=LT_SESSION_ID,
    actor_id=LT_ACTOR_ID
)

lt_session_manager = AgentCoreMemorySessionManager(
    agentcore_memory_config=lt_config,
    region_name="us-east-1"
)

# Create agent with long-term memory
lt_agent = Agent(
    system_prompt="You are a helpful assistant. Use all you know about the user to provide personalized responses.",
    session_manager=lt_session_manager
)

print("Agent created with long-term memory strategies")

### Test Long-Term Memory - Session 1

In [ ]:
# Share preferences
print("=== Session 1: Building Memory ===")

response1 = lt_agent("I like sushi with tuna")
print(f"\nUser: I like sushi with tuna")
print(f"Agent: {response1}")

response2 = lt_agent("I also enjoy pizza, especially margherita")
print(f"\nUser: I also enjoy pizza, especially margherita")
print(f"Agent: {response2}")

response3 = lt_agent("I'm vegetarian on weekdays but eat fish on weekends")
print(f"\nUser: I'm vegetarian on weekdays but eat fish on weekends")
print(f"Agent: {response3}")

### Test Cross-Session Memory

Create new session with same actor to test long-term memory persistence

In [ ]:
# Wait for memory consolidation (async process)
import time
print("Waiting 30 seconds for memory consolidation...")
time.sleep(30)
print("Consolidation complete")

In [ ]:
# Create new session with same actor
NEW_SESSION_ID = f"session_{datetime.now().strftime('%Y%m%d%H%M%S')}"

new_config = AgentCoreMemoryConfig(
    memory_id=LT_MEM_ID,
    session_id=NEW_SESSION_ID,  # New session
    actor_id=LT_ACTOR_ID  # Same actor
)

new_session_manager = AgentCoreMemorySessionManager(
    agentcore_memory_config=new_config,
    region_name="us-east-1"
)

new_agent = Agent(
    system_prompt="You are a helpful assistant. Use all you know about the user to provide personalized responses.",
    session_manager=new_session_manager
)

print(f"New session created: {NEW_SESSION_ID}")

In [ ]:
# Test if agent remembers from previous session
print("\n=== Session 2: Testing Memory Persistence ===")

response = new_agent("What should I have for lunch today? It's Wednesday.")
print(f"\nUser: What should I have for lunch today? It's Wednesday.")
print(f"Agent: {response}")
print("\n✅ Agent should suggest vegetarian options (weekday) and remember food preferences")

## Part 3: Inspect Memory Records

In [ ]:
# Retrieve memory records
import json

print("=== Stored Memory Records ===")

# Get all memory records for this actor
records = client.retrieve_memory_records(
    memory_id=LT_MEM_ID,
    actor_id=LT_ACTOR_ID,
    max_results=10
)

print(f"\nFound {len(records.get('memoryRecords', []))} memory records:\n")

for i, record in enumerate(records.get('memoryRecords', []), 1):
    print(f"{i}. Type: {record.get('memoryType')}")
    print(f"   Content: {record.get('content')[:100]}...")
    print(f"   Namespace: {record.get('namespace')}")
    print()

## Part 4: Cleanup (Optional)

In [ ]:
# Delete memories (uncomment to run)
# client.delete_memory(memory_id=MEM_ID)
# client.delete_memory(memory_id=LT_MEM_ID)
# print("Memories deleted")

## Key Takeaways

### Memory Types
- **Short-term**: Immediate context within session
- **Long-term**: Persistent knowledge across sessions

### Memory Strategies
- **Semantic**: Facts and knowledge
- **Preference**: User preferences and patterns
- **Summary**: Conversation narratives

### Key Concepts
- **Actor ID**: Identifies the user (persistent across sessions)
- **Session ID**: Identifies conversation instance
- **Memory ID**: Identifies the memory store
- **Namespaces**: Organize memories hierarchically

### Performance
- Extraction/consolidation: 20-40 seconds (async)
- Retrieval: ~200ms
- Compression: 89-95% vs full history